In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### 데이터 불러오기 : 자전거 대여량 수요 예측

In [ ]:
df = pd.read_csv('./datasets/bike_train.csv')
pd.set_option('display.max_columns', len(df.columns))
df.head()

주요 컬럼 설명
- datetime: hourly date + timestamp
- season: 1 = 봄, 2 = 여름, 3 = 가을, 4 = 겨울
- holiday: 1 = 토, 일요일의 주말을 제외한 국경일 등의 휴일, 0 = 휴일이 아닌 날
- workingday: 1 = 토, 일요일의 주말 및 휴일이 아닌 주중, 0 = 주말 및 휴일
- weather: 1 = 맑음, 약간 구름 낀 흐림 2 = 안개, 안개 + 흐림 3 = 가벼운 눈, 가벼운 비 + 천둥 4 = 심한 눈/비, 천둥/번개
- temp: 온도(섭씨)
- atemp: 체감온도(섭씨)
- humidity: 상대습도
- windspeed: 풍속
- casual: 사전에 등록되지 않는 사용자가 대여한 횟수
- registered: 사전에 등록된 사용자가 대여한 횟수
- count: 대여 횟수

자전거 대여 횟수(Y) 예측을 위해 X를 만들어 모델에 적용해보세요!

In [ ]:
# TODO : 데이터 전처리 자유롭게

In [ ]:
df.head(3)

In [ ]:
# 띄어쓰기 기준으로 칼럼 내용 내눠서 각 칼럼으로 저장
df['year'] = df['datetime'].apply(
    lambda x: x.split(' ')[0][0 : 4])

df['month'] = df['datetime'].apply(
    lambda x: x.split(' ')[0][5 : -3])

df['time'] = df['datetime'].apply(
    lambda x: x.split(' ')[1][ : 2])
pd.set_option('display.max_columns', len(df.columns))
df.head(3)

In [ ]:
# 테이블 컬럼 타입 확인
df.info()

In [ ]:
# 칼럼 타입 변경
df = df.astype({'year' : 'int64',
               'month' : 'int64',
               'time' : 'int64'})
df.info()

In [ ]:
check_df = df

print('rows x columns :', check_df.shape) # (행 개수, 열 개수)
non_null_cut_df = pd.DataFrame(check_df.count()).rename(columns = {0 : 'non_null_count'}).reset_index()# 칼럼별 비결측치 개수
info_df = pd.DataFrame(df.dtypes).rename(columns = {0 : 'dtype'}).reset_index() # 데이터 프레임 info
# 칼럼별 결측지 비율 확인
null_cnt_df = pd.DataFrame(check_df.isnull().sum()).rename(columns = {0 : 'null_count'}).reset_index()
null_cnt_df['null_ratio'] = round(null_cnt_df['null_count'] / len(check_df) * 100, 2)
# 합치기
non_null_cut_df = pd.merge(non_null_cut_df, info_df)
info_null_cnt_df = pd.merge(non_null_cut_df, null_cnt_df).rename(columns = {'index' : 'columns'})
info_null_cnt_df

In [ ]:
del df['datetime']

In [ ]:
df

### 모델 적용 / 평가

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler객체 생성
scaler = MinMaxScaler()
# MinMaxScaler 로 데이터 셋 변환. fit() 과 transform() 호출.
scaler.fit(df)
df_scaled = scaler.transform(df)

# transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
df_scaled_df = pd.DataFrame(data = df_scaled)
df_scaled_df

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 사이킷런의 mean_square_error() 를 이용하여 RMSE 계산
def rmse(y, pred):
    return np.sqrt(mean_squared_error(y, pred))

# MSE, RMSE, RMSLE 를 모두 계산
def evaluate_regr(y, pred):
    rmse_val = rmse(y, pred)
    # MAE 는 scikit learn의 mean_absolute_error() 로 계산
    mae_val = mean_absolute_error(y, pred)
    print('RMSE : {0:.3f}, MAE : {0:.3f}'.format(rmse_val, mae_val))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = df_scaled_df.drop([10], axis = 1, inplace = False).to_numpy()
y = df_scaled_df[10].to_numpy()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
pred = lr_reg.predict(X_test)

In [ ]:
evaluate_regr(y_test, pred)

In [ ]:
# 잘되지 않아서 뭔가 이것 저것 해보았지만 달라지는게 없었습니다...

In [ ]:
rmse(y_test, pred)

In [ ]:
mean_absolute_error(y_test, pred)

In [ ]:
pd.DataFrame(data = [y_test, pred])